In [16]:
%cd /Users/artus/Desktop/Projects/FastAPI/videome

c:\Users\artus\Desktop\Projects\FastAPI\videome


In [17]:
from app import db
from app.users.models import User
from cassandra.cqlengine.management import sync_table

In [20]:
db.get_session()
sync_table(User)

In [21]:
post_data = {
    "email": "mike@abc.com",
    "password": "abc123"
}

In [22]:
from pydantic import BaseModel, EmailStr, SecretStr, validator

In [23]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [24]:
UserLoginSchema(**post_data)

UserLoginSchema(email='mike@abc.com', password=SecretStr('**********'))

In [25]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr
        
    
    @validator("email")
    def email_available(cls, v, values, **kwargs):
        q = User.objects.filter(email=v)
        if q.count() != 0:
            raise ValueError("Email is not available")
        return v
        
    @validator("password_confirm")
    def passwords_match(cls, v, values, **kwargs):
        password = values.get('password')
        password_confirm = v
        if password != password_confirm:
            raise ValueError("Passwords do not match")
        return v

In [29]:
data = UserSignupSchema(email='abc@gmail.com', password='abc123', password_confirm='abc123')

In [31]:
data.json()

'{"email": "abc@gmail.com", "password": "**********", "password_confirm": "**********"}'

In [32]:
data.dict()

{'email': 'abc@gmail.com',
 'password': SecretStr('**********'),
 'password_confirm': SecretStr('**********')}

In [30]:
data.dict()['password'].get_secret_value()

'abc123'

In [33]:
dir(data.dict()['password'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__modify_schema__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_secret_value',
 'display',
 'get_secret_value',
 'max_length',
 'min_length',
 'validate']